# Deep Mailing - XGBoost Model - Calculado Correlacoes

O objetivo desse notebook é demonstrar a utilizacao do XGBoost para a criacao de arvores de decisão para a predição de CUPS em mailings.

Em primeiro lugar, definimos os imports que iremos usar...

In [ ]:
import xgboost
import numpy as np
import os
import sys
import logging
import gc
import pickle as pickle
import pandas as pd
import dateutil.parser as parser
import os.path
import math
from sklearn.metrics import accuracy_score,precision_score,recall_score
from scipy.stats import pearsonr
from datetime import datetime
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_tree
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams

Abaixo definimos os diretorios e nomes dos arquivos intermediarios.

In [ ]:
log_location = "../logs/"
arquivo_df_pickled_norm = "../intermediate/df.norm.pickle"
arquivo_df_pickled_norm_corr = "../intermediate/df.norm.corr.pickle"
arquivo_df_pickled_norm_train = "../intermediate/df.norm.train.pickle"
arquivo_df_pickled_norm_test = "../intermediate/df.norm.test.pickle"
arquivo_df_pickled_norm_train_x = "../intermediate/df.norm.train.x.pickle.npy"
arquivo_df_pickled_norm_train_y = "../intermediate/df.norm.train.y.pickle.npy"
arquivo_df_pickled_norm_test_x = "../intermediate/df.norm.test.x.pickle.npy"
arquivo_df_pickled_norm_test_y = "../intermediate/df.norm.test.y.pickle.npy"

Redefinimos o logger que iremos usar

In [ ]:
logger = logging.getLogger()
logging.basicConfig(format="%(asctime)-15s %(message)s",
                    level=logging.DEBUG,
                    filename=os.path.join(log_location,'correlations.log.' + datetime.now().strftime("%Y%m%d%H%M%S.%f") + '.log'))

Criamos uma função para imprimir tanto no log quanto no notebook...

In [ ]:
def print_log(msg):
    logging.debug(msg)
    print(msg)    

Carregamos para a memoria o arquivo normalizado e pickled que foi gerado no notebook de "Preparação de Dados".

In [ ]:
print_log("Carregando Pickling normalizado:{}".format(arquivo_df_pickled_norm))    
chamadas = pd.read_pickle(arquivo_df_pickled_norm)

Verificamos as dimensões do dataframe carregado.... E imprimimos uma amostra do dado que precisamos com apenas as colunas relevantes... Como podemos perceber o nosso modelo considera apenas colunas com valores booleanos (0 ou 1)

In [ ]:
chamadas.head(10)

In [ ]:
print_log(chamadas.shape)
chamadas.loc[:, 'NORM_CARTEIRA_W02':'NORM_LIGACOES'].head(10)

Para tratarmos as colunas de forma correta, precisamos eliminar os caracteres especiais das colunas, principalmente o espaco e o sinal de - para isso iremos rodar o codigo abaixo para tratar.

In [ ]:
cols = chamadas.columns
cols = cols.map(lambda x: x.replace(' ', '_').replace('-','menos'))
chamadas.columns = cols

Vimos q na realidade ter LIGACOES é fundamental para ter CUP...

In [ ]:
chamadas_calculo_correlacao = chamadas.loc[:, 'NORM_CARTEIRA_W02':'NORM_CUP'].query('NORM_CUP >= 0')

In [ ]:
print_log("Registros com CUP:{}".format(len(chamadas_calculo_correlacao)))

In [ ]:
chamadas_calculo_correlacao_cup = chamadas_calculo_correlacao.loc[:, 'NORM_CUP':'NORM_CUP'].as_matrix().ravel().astype(float)
print(chamadas_calculo_correlacao_cup)

In [ ]:
corr_campos_computados = { "coluna" : [], "corr": [] }

for coluna in chamadas_calculo_correlacao.columns.values:
    if coluna == 'NORM_CUP':
        continue
    chamadas_calculo_correlacao_coluna = chamadas_calculo_correlacao.loc[:, coluna:coluna].as_matrix().ravel().astype(float)
    corr = abs(pearsonr(chamadas_calculo_correlacao_coluna, chamadas_calculo_correlacao_cup)[0] * 100)
    corr_campos_computados['coluna'] = corr_campos_computados['coluna'] + [coluna]
    corr_campos_computados['corr'] = corr_campos_computados['corr'] + [corr]

corr = pd.DataFrame.from_dict(corr_campos_computados).sort_values('corr',ascending = False)
corr.head(20)

In [ ]:
colunas_finais = []
for coluna in corr.coluna:
    if coluna == 'NORM_TENTATIVAS':
        continue
    if coluna == 'NORM_LIGACOES':
        continue        
    if 'nan' in coluna:
        continue
    if 'DDD' in coluna:
        break
    colunas_finais = colunas_finais + [coluna]

colunas_finais = colunas_finais + ['NORM_TENTATIVAS', 'NORM_LIGACOES', 'NORM_CUP']
chamadas_novo = chamadas[colunas_finais]
chamadas_novo.head(10)

In [ ]:
chamadas_novo.to_pickle(arquivo_df_pickled_norm_corr)